In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pprint import pprint 
import matplotlib.dates as mpdates
from matplotlib import cm
from matplotlib import colors
from viz_utils import *
from ipywidgets import widgets, interactive
from sklearn.preprocessing import StandardScaler as SS
from sklearn.svm import SVR
sns.set()

In [ ]:
closed_deals = pd.read_csv('..\data\olist_closed_deals_dataset.csv', parse_dates=["won_date"])

customers = pd.read_csv('..\data\olist_customers_dataset.csv')

geolocation = pd.read_csv('..\data\olist_geolocation_dataset.csv')

marketing_qualified_leads = pd.read_csv('..\data\olist_marketing_qualified_leads_dataset.csv', parse_dates=["first_contact_date"])

order_items = pd.read_csv('..\data\olist_order_items_dataset.csv', parse_dates=["shipping_limit_date"])

order_payments = pd.read_csv('..\data\olist_order_payments_dataset.csv')

order_reviews = pd.read_csv('..\data\olist_order_reviews_dataset.csv', parse_dates=["review_creation_date", 'review_answer_timestamp'])

orders = pd.read_csv('..\data\olist_orders_dataset.csv', parse_dates=["order_purchase_timestamp", 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date'])

products = pd.read_csv('..\data\olist_products_dataset.csv')

sellers = pd.read_csv('..\data\olist_sellers_dataset.csv')

In [ ]:
color_pal=['blue','gold','violet', 'green', 'black', 'cyan', 'orange', 'brown', 'gray',
                                                         'orange', 'purple', 'red']
ref=np.log(1)

### población Brasil

In [ ]:
popul = pd.read_html('https://es.wikipedia.org/wiki/Demograf%C3%ADa_de_Brasil#:~:text=La%20distribuci%C3%B3n%20de%20la%20poblaci%C3%B3n,Oeste%20y%20en%20el%20Norte.', header=0)[2]
popul = popul.sort_values('Pob. (2020)', ascending=False)

In [ ]:
plt.figure(figsize = (12,6))
sns.barplot(x='Sigla', y='Pob. (2020)', data=popul)
plt.title('Habitantes por estado', fontsize=40)
plt.xlabel('Código de estado')
plt.ylabel('Habitantes')
plt.show()

## Customers Dataset

### Mismos usuarios con diferentes ID

In [ ]:
custom=['Customer', 'Unique customer']
cust_val=[customers.customer_unique_id.value_counts().sum(), len(customers.customer_unique_id.value_counts())]

In [ ]:
graph=plt.bar(custom, cust_val, color=['green','red'], alpha=0.5)
i = 0
for p in graph:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    plt.text(x+width/2,
             y+height*1.01,
             str(cust_val[i]),
             ha='center',
             weight='bold')
    i+=1
plt.title('Registro de clientes y clientes únicos', fontsize=20)
plt.ylabel('Cantidad de registros')
plt.xticks(custom);

In [ ]:
unicos=[]
valores=[]
for i in range(7):
    if (customers.customer_unique_id.value_counts()>1)[i]:
        unicos.append((customers.customer_unique_id.value_counts()>1).index[i])
        valores.append(customers.customer_unique_id.value_counts()[i])

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(unicos, valores, palette="Blues_r")
plt.xticks(rotation=45)
plt.title('Top mismos usuarios con diferentes cuentas', fontsize=30)
plt.xlabel('Customer Unique ID')
plt.ylabel('Cantidad de cuentas diferentes')
plt.show()

In [ ]:
sns.barplot(x=customers.customer_state.value_counts().index , y=customers.customer_state.value_counts())
plt.xlabel('Código de estado')
plt.ylabel('Número de clientes')
plt.title('Clientes por estado', fontsize=40);

In [ ]:
rep_cust=customers.customer_state.value_counts()/customers.customer_state.value_counts().sum()
goals=[]
labels=[]
sum=0
for i in range(len(rep_cust)):
    if rep_cust[i] >= 0.03:
        sum+=rep_cust[i]
        goals.append(rep_cust[i])
        labels.append(rep_cust.index[i])
    else:
        break
goals.append(1-sum)
labels.append('OTROS')
plt.figure(figsize=(8,8))
plt.pie(goals, autopct="%0.1f %%")
plt.legend(labels, bbox_to_anchor=(0, 1), title='Código estado')
plt.title('Distribución de los clientes por estado', fontsize=40)
plt.show()

## Geolocalization

### Creando archivo de localización por estado

In [ ]:
stt = pd.read_html('https://es.wikipedia.org/wiki/Unidad_federativa_de_Brasil', header=0)[0]

In [ ]:
entidades=[]
for i in range(len(('BR-' + geolocation.geolocation_state.unique()))):
    entidades.append(stt[stt['Código ISO']==(('BR-' + geolocation.geolocation_state.unique())[i])]['Unidad federativa'].values[0])

In [ ]:
latitud=[]
longitud=[]
for i in range(len(geolocation.geolocation_state.unique())):
    latitud.append(geolocation[geolocation.geolocation_state==geolocation.geolocation_state.unique()[i]].geolocation_lat.mean())
    longitud.append(geolocation[geolocation.geolocation_state==geolocation.geolocation_state.unique()[i]].geolocation_lng.mean())
state_location = pd.DataFrame(list(zip('BR-' + geolocation.geolocation_state.unique(), geolocation.geolocation_state.unique(), entidades ,latitud, longitud)), columns = ['State_Code', 'State', 'State_name','Lat', 'Lon'])
state_location.to_csv('..\data\olist_state_location.csv')

## Order_items

### Artículos vendidos y órdenes realizadas

In [ ]:
prod=['Productos vendidos', 'Órdenes realizadas']
prod_val=[order_items.order_id.value_counts().sum(), len(order_items.order_id.unique())]

In [ ]:
graph=plt.bar(prod, prod_val, color=['blue','red'], alpha=0.5)
i = 0
for p in graph:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    plt.text(x+width/2,
             y+height*1.01,
             str(prod_val[i]),
             ha='center',
             weight='bold')
    i+=1
plt.title('Productos vendidos y órdenes realizadas', fontsize=20)
plt.ylabel('Cantidad de registros')
plt.xticks(prod);

In [ ]:
unicos=[]
valores=[]
for i in range(10):
    if (order_items.order_id.value_counts()>1)[i]:
        unicos.append((order_items.order_id.value_counts()>1).index[i])
        valores.append(order_items.order_id.value_counts()[i])

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(unicos, valores, palette="Blues_r")
plt.xticks(rotation=45)
plt.title('Top artículos por órden', fontsize=30)
plt.xlabel('Order ID')
plt.ylabel('Cantidad de productos')
plt.show()

In [ ]:
# KPI Top 10 ingresos por categoria
KPI_categoria_ingreso = order_items.groupby(['product_id']).sum()['price'].reset_index()
KPI_categoria_ingreso = KPI_categoria_ingreso.merge(products, on='product_id', copy=False)
KPI_categoria_ingreso = KPI_categoria_ingreso.drop(columns=['product_name_lenght', 
                                                            'product_description_lenght', 
                                                            'product_photos_qty', 
                                                            'product_weight_g', 
                                                            'product_length_cm', 
                                                            'product_height_cm',
                                                            'product_width_cm'])
KPI_categoria_ingreso = KPI_categoria_ingreso.merge(categoryName, on='product_category_name', copy=False)
KPI_categoria_ingreso = KPI_categoria_ingreso.groupby('product_category_name_english').sum().sort_values('price', ascending=False).reset_index().head(10)

# Grafico
ax=sns.barplot(x=KPI_categoria_ingreso['product_category_name_english'], y=KPI_categoria_ingreso['price'])
ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)
plt.title('Top 10 Ingresos por categoria', fontsize=40)
plt.xlabel('Categorias')
plt.ylabel('Total de ingreso')
plt.show()

### KPI Embrace

In [ ]:
income=[]
for i in range(len(KPI_categoria_ingreso)):
    if KPI_categoria_ingreso.price[i]/KPI_categoria_ingreso.price[0] >= 0.5:
        income.append(KPI_categoria_ingreso.price[i]/KPI_categoria_ingreso.price[0])
    else: break
KPI_categoria_ingreso = KPI_categoria_ingreso.groupby('product_category_name_english').sum().sort_values('price', ascending=False).reset_index().head(i)

# Grafico
ax=sns.barplot(x=KPI_categoria_ingreso['product_category_name_english'], y=income)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 75)
plt.title('KPI Embrace', fontsize=20)
plt.xlabel('Categorias')
plt.ylabel('Valor KPI')
plt.show()

## Order_payments

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
single_countplot(order_payments, x='payment_type', ax=ax)
plt.title('Distribución métodos de pago', fontsize=30)
plt.show()

In [ ]:
pay_meth=[]
pay_seq=[]
for i in range(1, order_payments.payment_type.value_counts().sum()+1, 1):
    if len(order_payments[order_payments.payment_sequential>=i].payment_type.value_counts()) == 1:
        pay_meth.append(1)
        pay_seq.append(i)
        break
    elif len(order_payments[order_payments.payment_sequential>=i].payment_type.value_counts()) > 1:
        pay_meth.append(len(order_payments[order_payments.payment_sequential>=i].payment_type.value_counts()))
        pay_seq.append(i)

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(pay_seq, pay_meth, palette="Blues_r")
plt.title('Métodos de pago empleados por cantidad de pagos realizados por compra', fontsize = 20)
plt.xlabel('Cantidad de pagos por compra')
plt.ylabel('Cantidad de métodos de pago utilizados')
plt.show()

In [ ]:
KPI_payment_method = pd.DataFrame()
KPI_payment_method['order_id'] = orders['order_id']
KPI_payment_method['date'] = orders['order_purchase_timestamp']
KPI_payment_method['date'] = KPI_payment_method['date'].dt.strftime('%Y-%m')
KPI_payment_method = KPI_payment_method.merge(order_payments, on='order_id', copy=False)
KPI_payment_method = KPI_payment_method.drop(columns=['order_id','payment_sequential','payment_installments','payment_value'])
KPI_payment_method = KPI_payment_method.value_counts().unstack().reset_index()

In [ ]:
for elem in order_payments.payment_type.unique():
    sns.lineplot(x = 'date', y = elem, data = KPI_payment_method)
sns.set(rc = {'figure.figsize':(10,6)})
plt.title('Métodos de pago a lo largo del tiempo', fontsize=30)
plt.xlabel('Fecha (aaaa,mm)')
plt.ylabel('Número de transacciones pagadas')
plt.xticks(rotation=45)
plt.legend(labels=order_payments.payment_type.unique(), title='Métodos de pago', 
           fontsize = 'medium', title_fontsize = "12")
plt.show()

## Representatibilidad de los compradores en el país

In [ ]:
rep_cust=customers.customer_state.value_counts()/customers.customer_state.value_counts().sum()
goals=[]
labels=[]
sum=0
for i in range(len(rep_cust)):
    if rep_cust[i] >= 0.03:
        sum+=rep_cust[i]
        goals.append(rep_cust[i])
        labels.append(rep_cust.index[i])
    else:
        break
goals.append(1-sum)
labels.append('OTROS')
plt.figure(figsize=(8,8))
plt.pie(goals, autopct="%0.1f %%")
plt.legend(labels, bbox_to_anchor=(0, 1))
plt.show()

## Representatibilidad de los vendedores en el país

In [ ]:
sns.barplot(x=sellers.seller_state.value_counts().index , y=sellers.seller_state.value_counts())
plt.xlabel('Código de estado')
plt.ylabel('Número de vendedores')
plt.title('Vendedores por estado', fontsize=40);

In [ ]:
rep_sell=sellers.seller_state.value_counts()/sellers.seller_state.value_counts().sum()
goals=[]
labels=[]
sum=0
for i in range(len(rep_cust)):
    if rep_cust[i] >= 0.03:
        sum+=rep_sell[i]
        goals.append(rep_sell[i])
        labels.append(rep_sell.index[i])
    else:
        break
goals.append(1-sum)
labels.append('OTROS')
plt.figure(figsize=(8,8))
plt.pie(goals, autopct="%0.1f %%")
plt.legend(labels, bbox_to_anchor=(0, 1), title='Código estado')
plt.show()

### KPI Contundencia

KPI_C=delivered/orders

In [ ]:
year_cont=[]
cont=[]
for i in range (int(orders.order_approved_at.describe()[4].strftime("%Y")), int(orders.order_approved_at.describe()[5].strftime("%Y"))+1, 1):
    year_cont.append(i)
    cont.append(orders[orders.order_approved_at.dt.year==i].order_status.value_counts()[0]/orders[orders.order_approved_at.dt.year==i].order_status.value_counts().sum())
graph=plt.bar(year_cont, cont, color=['blue','red','green'], alpha=0.5)
i = 0
for p in graph:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    plt.text(x+width/2,
             y+height*1.01,
             str(round(cont[i], 2)),
             ha='center',
             weight='bold')
    i+=1
plt.title('KPI contundencia')
plt.xticks(year_cont);    

### Orders

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
single_countplot(df_orders, x='order_status', ax=ax)
plt.title('Distribución del estatus de la orden', fontsize=35)
plt.xlabel('Estatus de la orden')
plt.ylabel('Numero de órdenes')
plt.show()

### Products

In [ ]:
## df multiindice
df = pd.merge(order_items[['seller_id','product_id']],products[['product_id','product_category_name']],on='product_id').groupby(['product_category_name',
                                                                                                                            'seller_id']).count().sort_values('product_id',ascending=False)
## df con un indice

df = df.reset_index(level='seller_id').sort_index()
df_top_categoria = pd.DataFrame()
for i in df.index.unique():
    df_top_categoria = pd.concat([df_top_categoria,df[df.index == i].sort_values(by='product_id',ascending=False).head(10)])
    
categoria = widgets.Dropdown(
    options=['All'] + list(df_top_categoria.index.unique()),
    value='All',
    description='categoria:',
)
def plotit(categoria):
    plt.bar(df_top_categoria[df_top_categoria.index == categoria]['seller_id'],df_top_categoria[df_top_categoria.index == categoria]['product_id'])
    plt.xticks(rotation = 90) 
    plt.show()


### KPI Latencia (envío)

KPI_L_S = 1-((Ln('delta' time to carrier deliver +1))/Ln('delta' max time passed))

### Anual

In [ ]:
year_cont=[]
empaq=[]
for i in range (int(orders.order_approved_at.describe()[4].strftime("%Y")), int(orders.order_approved_at.describe()[5].strftime("%Y"))+1, 1):
    year_cont.append(i)
    val=(1-((np.log(((orders[orders.order_approved_at.dt.year==i].order_delivered_carrier_date - orders[orders.order_approved_at.dt.year==i].order_approved_at).map(mpdates.date2num)+1)))/(np.log(np.max((orders[orders.order_approved_at.dt.year==i].order_delivered_carrier_date - orders[orders.order_approved_at.dt.year==i].order_approved_at).map(mpdates.date2num)))))).mean()
    if type(val) == type(ref):
        empaq.append(val)
    else:
        empaq.append(0)
graph=plt.bar(year_cont, empaq, color=['blue','red','green'], alpha=0.5)
i = 0
for p in graph:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    plt.text(x+width/2,
             y+height*1.01,
             str(round(empaq[i], 2)),
             ha='center',
             weight='bold')
    i+=1
plt.title('KPI Latencia A (anual)')
plt.xlabel('Mes')
plt.ylabel('KPI')
plt.xticks(year_cont);    

### Mensual

In [ ]:
for i in range (int(orders.order_approved_at.describe()[4].strftime("%Y")), int(orders.order_approved_at.describe()[5].strftime("%Y"))+1, 1):
    delivr_month=[]
    month=[]
    numbers=[]
    for j in range (int(np.min(orders3[orders3.order_delivered_carrier_date.dt.year==i].order_delivered_carrier_date.dt.month)), int(np.max(orders3[orders3.order_delivered_carrier_date.dt.year==i].order_delivered_carrier_date.dt.month))+1, 1):
        numbers.append(j)
        if j==1:
            month.append('Enero')
        elif j==2:
            month.append('Febrero')
        elif j==3:
            month.append('Marzo')
        elif j==4:
            month.append('Abril')
        elif j==5:
            month.append('Mayo')
        elif j==6:
            month.append('Junio')
        elif j==7:
            month.append('Julio')
        elif j==8:
            month.append('Agosto')
        elif j==9:
            month.append('Septiembre')
        elif j==10:
            month.append('Octubre')
        elif j==11:
            month.append('Noviembre')
        elif j==12:
            month.append('Diciembre')
        val=(1-((np.log(((orders3[(orders3.order_approved_at.dt.year==i) & (orders3.order_approved_at.dt.month==j)].order_delivered_carrier_date - orders3[(orders3.order_approved_at.dt.year==i) & (orders3.order_approved_at.dt.month==j)].order_approved_at).map(mpdates.date2num)+1)))/(np.log(np.max((orders3[orders3.order_approved_at.dt.year==i].order_delivered_carrier_date - orders3[orders3.order_approved_at.dt.year==i].order_approved_at).map(mpdates.date2num)))))).mean()
        if type(val) == type(ref):
            delivr_month.append(val)
        else:
            delivr_month.append(0)
    pal=[]
    for elem in numbers:
        pal.append(color_pal[elem-1])
    print('-----------------------------------------------------', 'Año', i, '-----------------------------------------------------')
    plt.figure(figsize=(9,5))
    graph=plt.bar(month, delivr_month, alpha=0.5, color=pal)
    k = 0
    for p in graph:
        width = p.get_width()
        height = p.get_height()
        x, y = p.get_xy()
        plt.text(x+width/2,
                y+height*1.01,
                str(round(delivr_month[k], 2)),
                ha='center',
                weight='bold')
        k+=1
    plt.title('KPI Latencia A')
    plt.xticks(month, rotation = 45)
    plt.xlabel('Mes')
    plt.ylabel('KPI')
    plt.show()

### KPI Latencia (entrega)

KPI_L_D = 1-((Ln('delta' time to carrier deliver +1))/Ln('delta' max time passed))

### Anual

In [ ]:
year_cont=[]
delivr=[]
for i in range (int(orders.order_approved_at.describe()[4].strftime("%Y")), int(orders.order_approved_at.describe()[5].strftime("%Y"))+1, 1):
    year_cont.append(i)
    val=(1-((np.log(((orders[orders.order_delivered_carrier_date.dt.year==i].order_delivered_customer_date - orders[orders.order_delivered_carrier_date.dt.year==i].order_delivered_carrier_date).map(mpdates.date2num)+1)))/(np.log(np.max((orders[orders.order_delivered_carrier_date.dt.year==i].order_delivered_customer_date - orders[orders.order_delivered_carrier_date.dt.year==i].order_delivered_carrier_date).map(mpdates.date2num)))))).mean()
    if type(val) == type(ref):
        delivr.append(val)
    else:
        delivr.append(0)
graph=plt.bar(year_cont, delivr, color=['blue','red','green'], alpha=0.5)
i = 0
for p in graph:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    plt.text(x+width/2,
             y+height*1.01,
             str(round(delivr[i], 2)),
             ha='center',
             weight='bold')
    i+=1
plt.title('KPI Latencia B (anual)')
plt.xlabel('Mes')
plt.ylabel('KPI')
plt.xticks(year_cont)
plt.show()

### Mensual

In [ ]:
for i in range (int(orders.order_approved_at.describe()[4].strftime("%Y")), int(orders.order_approved_at.describe()[5].strftime("%Y"))+1, 1):
    delivr_month=[]
    month=[]
    numbers=[]
    for j in range (int(np.min(orders3[orders3.order_delivered_carrier_date.dt.year==i].order_delivered_carrier_date.dt.month)), int(np.max(orders3[orders3.order_delivered_carrier_date.dt.year==i].order_delivered_carrier_date.dt.month))+1, 1):
        numbers.append(j)
        if j==1:
            month.append('Enero')
        elif j==2:
            month.append('Febrero')
        elif j==3:
            month.append('Marzo')
        elif j==4:
            month.append('Abril')
        elif j==5:
            month.append('Mayo')
        elif j==6:
            month.append('Junio')
        elif j==7:
            month.append('Julio')
        elif j==8:
            month.append('Agosto')
        elif j==9:
            month.append('Septiembre')
        elif j==10:
            month.append('Octubre')
        elif j==11:
            month.append('Noviembre')
        elif j==12:
            month.append('Diciembre')
        val=(1-((np.log(((orders3[(orders3.order_delivered_carrier_date.dt.year==i) & (orders3.order_delivered_carrier_date.dt.month==j)].order_delivered_customer_date - orders3[(orders3.order_delivered_carrier_date.dt.year==i) & (orders3.order_delivered_carrier_date.dt.month==j)].order_delivered_carrier_date).map(mpdates.date2num)+1)))/(np.log(np.max((orders3[orders3.order_delivered_carrier_date.dt.year==i].order_delivered_customer_date - orders3[orders3.order_delivered_carrier_date.dt.year==i].order_delivered_carrier_date).map(mpdates.date2num)))))).mean()
        if type(val) == type(ref):
            delivr_month.append(val)
        else:
            delivr_month.append(0)
    pal=[]
    for elem in numbers:
        pal.append(color_pal[elem-1])
    print('-----------------------------------------------------', 'Año', i, '-----------------------------------------------------')
    plt.figure(figsize=(9,5))
    graph=plt.bar(month, delivr_month, alpha=0.5, color=pal)
    k = 0
    for p in graph:
        width = p.get_width()
        height = p.get_height()
        x, y = p.get_xy()
        plt.text(x+width/2,
                y+height*1.01,
                str(round(delivr_month[k], 2)),
                ha='center',
                weight='bold')
        k+=1
    plt.title('KPI Latencia B (mensual)')
    plt.xticks(month, rotation = 45)
    plt.xlabel('Mes')
    plt.ylabel('KPI')
    plt.show()

### Ventas anuales

In [ ]:
year_cont=[]
cont=[]
for i in range (int(orders.order_approved_at.describe()[4].strftime("%Y")), int(orders.order_approved_at.describe()[5].strftime("%Y"))+1, 1):
    year_cont.append(i)
    cont.append(ventas_pagos[ventas_pagos.order_approved_at.dt.year==i].payment_value.sum())
graph=plt.bar(year_cont, cont, color=['blue','red','green'], alpha=0.5)
i = 0
for p in graph:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    plt.text(x+width/2,
             y+height*1.01,
             str(round(cont[i], 2)),
             ha='center',
             weight='bold')
    i+=1
plt.title('Ventas anuales', fontsize=35)
plt.xlabel('Año')
plt.ylabel('Ventas totales (moneda local)')
plt.xticks(year_cont);   

### Ventas anuales extracto mensual

In [ ]:
year_cont=[]
cont=[]
for i in range (int(orders.order_approved_at.describe()[4].strftime("%Y")), int(orders.order_approved_at.describe()[5].strftime("%Y"))+1, 1):
    year_cont.append(i)
    cont.append(ventas_pagos[ventas_pagos.order_approved_at.dt.year==i].payment_value.sum()/(np.max(ventas_pagos[ventas_pagos.order_approved_at.dt.year==i].order_approved_at.dt.month)-np.min(ventas_pagos[ventas_pagos.order_approved_at.dt.year==i].order_approved_at.dt.month)+1))
graph=plt.bar(year_cont, cont, color=['blue','red','green'], alpha=0.5)
i = 0
for p in graph:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    plt.text(x+width/2,
             y+height*1.01,
             str(round(cont[i], 2)),
             ha='center',
             weight='bold')
    i+=1
plt.title('Ventas anuales (extracto mensual)', fontsize=25)
plt.xlabel('Año')
plt.ylabel('Ventas totales (moneda local)')
plt.xticks(year_cont);  

### KPI Sales Performance

In [ ]:
y_cont=[1]
for i in range(len(cont)-1):
    y_cont.append(cont[i+1]/cont[i])

In [ ]:
plt.scatter(year_cont, np.log(y_cont)+1)
plt.plot(year_cont, np.log(y_cont)+1, color='green', alpha=0.3)
plt.plot([min(year_cont), max(year_cont)+2], [1, 1], color='red', alpha=0.7)
plt.xticks(year_cont)
plt.xlim(min(year_cont),max(year_cont)+0.5)
plt.title('KPI: Sales Performance (SP)', fontsize=25)
plt.xlabel('Año evaluado')
plt.ylabel('Valor KPI')
plt.show()

# Modelo ML SVR tendencia uso tarjta de crédito como método de pago

In [ ]:
KPI_payment_method.date=KPI_payment_method.date.map(mpdates.date2num)
X=KPI_payment_method.iloc[:23,0:1].values
y=KPI_payment_method.iloc[:23,2:3].valuessc_X= SS()
sc_y= SS()
X=sc_X.fit_transform(X)
y= sc_y.fit_transform(y.reshape(-1,1))
regresion = SVR(kernel= 'rbf')
regresion.fit(X, y)
X1= sc_X.inverse_transform(X)
y1= sc_y.inverse_transform(y)
X_grid = np.arange(min(X), 3.7, 0.1)
X_grid = X_grid.reshape(len(X_grid), 1)
y_pred_grid=regresion.predict(X_grid)
y_pred_grid = y_pred_grid.reshape(len(y_pred_grid), 1)

In [ ]:
fig, ax = plt.subplots()
ax.plot(sc_X.inverse_transform(X_grid), sc_y.inverse_transform(y_pred_grid), color = "blue", label='Línea de tendencia')
ax.set_xlabel('Date')
plt.scatter(X1, y1, color = "red", label='Valor real')
ax.xaxis.set_major_formatter(mpdates.DateFormatter('%Y-%m'))
fig.autofmt_xdate(rotation=0)
plt.title('Modelo de ML (SVR) y predicción', fontsize=30)
plt.xlabel('Fecha (aaaa-mm)')
plt.ylabel('Clintes que utilizan "crdit-card"')
plt.legend()
plt.show()

# Modelo MLR predicción precios de envío

In [ ]:
master_join = pd.merge(order_items, orders, how='left', left_on=order_items['order_id'], right_on=orders['order_id'])
master_join.drop(['key_0'], axis=1, inplace=True)
master_join = pd.merge(master_join, products, how='left', left_on=master_join['product_id'], right_on=products['product_id'])
master_join.drop(['key_0'], axis=1, inplace=True)
master_join = pd.merge(master_join, sellers, how='left', left_on=master_join['seller_id'], right_on=sellers['seller_id'])
master_join.drop(['key_0'], axis=1, inplace=True)
master_join = pd.merge(master_join, customers, how='left', left_on=master_join['customer_id'], right_on=customers['customer_id'])
master_join.drop(['key_0'], axis=1, inplace=True)
master_join = pd.merge(master_join, state_location, how='left', left_on=master_join['customer_state'], right_on=state_location['state'])
master_join.drop(['key_0'], axis=1, inplace=True)
master_join['customer_latitud']=master_join['latitud']
master_join['customer_longitud']=master_join['longitud']
master_join['customer_state']=master_join['state']
master_join.drop(['latitud', 'longitud', 'state'], axis=1, inplace=True)
master_join = pd.merge(master_join, state_location, how='left', left_on=master_join['seller_state'], right_on=state_location['state'])
master_join.drop(['key_0'], axis=1, inplace=True)
master_join['seller_latitud']=master_join['latitud']
master_join['seller_longitud']=master_join['longitud']
master_join['seller_state']=master_join['state']
master_join.drop(['latitud', 'longitud', 'state'], axis=1, inplace=True)
master_join['distance']=np.sqrt(np.power(master_join.customer_latitud - master_join.seller_latitud, 2) + 
                                np.power(master_join.customer_longitud - master_join.seller_longitud, 2))
master_join.drop(['seller_longitud', 'seller_latitud', 'customer_longitud', 'customer_latitud', 'customer_city',
                  'customer_zip_code_prefix', 'shipping_limit_date', 'order_id_y', 'order_status',
                  'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date',
                  'order_delivered_customer_date', 'order_estimated_delivery_date', 'product_id_y',
                  'product_name_lenght', 'product_category_name', 'product_name_lenght', 'product_description_lenght',
                  'product_photos_qty', 'seller_id_y', 'seller_zip_code_prefix', 'customer_id_y'], axis=1, inplace=True)
master_join.drop(['order_id_x', 'order_item_id', 'product_id_x', 'seller_id_x', 'customer_id_x',
                 'seller_city', 'seller_state', 'customer_unique_id', 'customer_state'], axis=1, inplace=True)
master_join.distance = round(master_join.distance, 2)
master_join.product_weight_g = round(master_join.product_weight_g, 2)
master_join.product_length_cm = round(master_join.product_length_cm, 2)
master_join.product_height_cm = round(master_join.product_height_cm, 2)
master_join.product_width_cm = round(master_join.product_width_cm, 2)
master_join = master_join.reset_index()
master_join.drop(['level_0', 'index'], axis=1, inplace=True)

In [ ]:
dataset=pd.read_csv('master_join.csv')
X=dataset.iloc[:,3:].values
y=dataset.iloc[:,2:3].values
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2, random_state=0)
from sklearn.linear_model import LinearRegression as LR
regresion=LR()
regresion.fit(X_train, y_train)
y_pred_MLR=regresion.predict(X_test)
X_opt=X_test
X_opt=np.append(arr= np.ones((22527,1)).astype(int), values=X_test, axis=1)
regresion_OLS = sm.OLS(endog = y_test, exog = X_opt.tolist()).fit()

In [ ]:
regresion_OLS.summary()